## Task 1: Working with a dataset with categorical features

In Assignment 1, we didn't have to do much preprocessing, because all the features in the two datasets were numerical. (Actually, in the second dataset, we removed all non-numerical features.) In this assignment, we'll instead consider how to deal with non-numerical features.

We'll use the famous Adult dataset. This is a binary classification task, where our task is to predict whether an American individual earns more than $50,000 a year, given a number of numerical and categorical features. (The dataset was extracted from a 1994 census database.)

### Step 1. Reading the data

Please download the two CSV files, the training set and the test set, and save them into your working directory This is the official train/test split defined by the people who created the dataset. It's the same data as in the the public distribution, except that we converted the format into a standard CSV format.

1. Write code to read the CSV file, for instance by using Pandas as in Assignment 1. 
2. Then split the data into an input part X and an output part Y. The output variable, which the classifier will predict, is called target.





In [ ]:
# import required libs
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon

In [ ]:
# Import csv file for Training data
train = pd.read_csv('adult_train.csv')

# Import csv file for Test data
test = pd.read_csv('adult_test.csv')

# Shuffle the dataset.
train_shuffled = train.sample(frac=1.0, random_state=0)
test_shuffled = test.sample(frac=1.0, random_state=0)

# Split into input part X and output part Y.
Xtrain = train_shuffled.drop('target', axis=1)
Ytrain = train_shuffled['target']
Xtest = test_shuffled.drop('target', axis=1)
Ytest = test_shuffled['target']

In [ ]:
Xtrain.head(3)

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
22278,49,Local-gov,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States
8950,49,Private,HS-grad,9,Divorced,Other-service,Not-in-family,Black,Female,0,0,40,United-States
7838,31,Private,Prof-school,15,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,50,United-States


### Step 2: Encoding the features as numbers.

If you look at the data, you will note that it contains several features with categorical values, such as workclass, education etc. All scikit-learn models work with numerical data internally; this means that the categorical features need to be converted to numbers. The most straightforward way to carry out such a conversion is to use one-hot encoding of the features, also known as dummy variables in statistics. In this approach, we define one new column for each observed value of the feature.

Scikit-learn includes a number of tools that can do one-hot encoding of categorical features and we'll see how to use one of them, the DictVectorizer. An alternative approach that is a bit more Pandas-friendly and gives more low-level control is to use the recently introduced ColumnTransformer; if you're interested, you can read an introduction to this approach here. We won't use a ColumnTransformer here because it will make Task 3 in this assignment a bit too annoying to solve.

The DictVectorizer is used when we store our features as named attributes in dictionaries. For instance, we could represent one individual from the Adult dataset as follows:

    {'age': 44,
    'workclass': 'Private',
    'education': 'Some-college',
    'education-num': 10,
    'marital-status': 'Married-civ-spouse',
    'occupation': 'Machine-op-inspct',
    'relationship': 'Husband',
    'race': 'Black',
    'sex': 'Male',
    'capital-gain': 7688,
    'capital-loss': 0,
    'hours-per-week': 40,
    'native-country': 'United-States'}

Pandas includes a utility to convert a DataFrame into a list of dictionaries:

    dicts_for_my_training_data = my_training_data.to_dict('records')

Then make a DictVectorizer and apply it, writing something like the following:

    dv = DictVectorizer()
    X_train_encoded = dv.fit_transform(dicts_for_my_training_data)

The method fit_transform will first call fit, which as usual is the "training" method. For a DictVectorizer, "training" consists of building the mapping from categories to column positions. Then, the transform method will be called, which converts the data into a matrix.

Now that you have a numerical representation of the data, you can compute a cross-validation accuracy for the training set using one of the classifiers you explored in Programming Assignment 1.

To handle the test data, you just call transform, because this time the vectorizer does not need to be "trained." Use this to compute the accuracy on the test set.

    X_test_encoded = dv.transform(dicts_for_my_test_data)

transform the data into dicts

In [ ]:
# convert training data to dict - creates a list of dicts 
# where each row's (person's) information is gathered in one dict, 
dicts_Xtrain = Xtrain.to_dict('records')
dicts_Xtest = Xtest.to_dict('records')

# show first 2 person's informations
dicts_Xtrain[:1] 

[{'age': 49,
  'workclass': 'Local-gov',
  'education': 'HS-grad',
  'education-num': 9,
  'marital-status': 'Married-civ-spouse',
  'occupation': 'Transport-moving',
  'relationship': 'Husband',
  'race': 'White',
  'sex': 'Male',
  'capital-gain': 0,
  'capital-loss': 0,
  'hours-per-week': 40,
  'native-country': 'United-States'}]

Define the dict vectorizer and transform X variables for train and test with one-hot encoding

In [ ]:
# create DictVectorizer 
dv = DictVectorizer()

# one-hot encode the list with all dicts for both train and test
Xtrain_encoded = dv.fit_transform(dicts_Xtrain)
Xtest_encoded = dv.transform(dicts_Xtest)

Get dummy performance

In [ ]:
# Baseline with dummy classifier
dummy = DummyClassifier(strategy='most_frequent')
# train
dummy.fit(Xtrain_encoded, Ytrain)
# cross validation on train data
dummy_score_train = cross_val_score(dummy, Xtrain_encoded, Ytrain).mean()
# accuracy on test data
dummy_score_test = accuracy_score(Ytest, dummy.predict(Xtest_encoded))

print("Dummy Classifier")
print("Average Train score:",dummy_score_train)
print("Average Test score:",dummy_score_test)

Dummy Classifier
Average Train score: 0.7591904454179904
Average Test score: 0.7637737239727289


Choosing a model from previous assignment 1: Decision Tree Classifier

In [ ]:
# Decision Tree Classifier
dtc = DecisionTreeClassifier() 
# train
dtc.fit(Xtrain_encoded, Ytrain)
# cross validation on train data
dtc_score_train = cross_val_score(dtc, Xtrain_encoded, Ytrain).mean()
# accuracy on test data
dtc_score_test = accuracy_score(Ytest, dtc.predict(Xtest_encoded))

print("Decision Tree Classifier")
print("Average Train score:",dtc_score_train)
print("Average Test score:",dtc_score_test)

Decision Tree Classifier
Average Train score: 0.8199381099830202
Average Test score: 0.8180087218229839


### Step 3. Combining the steps.

In the example above, we first transformed the list of dictionaries into a numerical matrix, and then we used this matrix when training the classifier. A separate preprocessing step was carried out for the test set.

In machine learning setups, we often use long chains of preprocessing steps. The one-hot encoding is one example, and other such steps might be scaling, feature selection, imputation of missing values, etc. As you can imagine, keeping track of the preprocessing steps can be tedious and error-prone, so it makes sense to handle such preprocessing chains automatically.

A Pipeline consists of a sequence of scikit-learn modules. The most convenient way to build a Pipeline is to use the utility function make_pipeline. For instance, to build a pipeline consisting of a vectorization step and then a decision tree classifier, we could write

    from sklearn.pipeline import make_pipeline
    
    pipeline = make_pipeline(
    DictVectorizer(),
    DecisionTreeClassifier() )
    
The Pipeline can be treated as any classifier: we can call fit and predict as usual. Concretely, when we call fit on a Pipeline, it will in turn call fit_transform on all intermediate steps and then fit on the final step. When we call predict, transform will be called on the intermediate steps and then predict on the final step.

Build a pipeline that includes the classifier that you selected previously, and make sure that it works.

Create pipeline including steps: Dictvectorize, StandardScaler, SelectKBest and estimator DecisionTreeClassifier

In [ ]:
pipeline = make_pipeline(
    DictVectorizer(),
    StandardScaler(with_mean=False),
    SelectKBest(k=100),
    DecisionTreeClassifier()) 

Adding hyperparameters that can be tested

In [ ]:
# specifying parameters for the models
param_grid = {'decisiontreeclassifier__criterion':['gini', 'entropy'], 
                'decisiontreeclassifier__splitter':['best', 'random'],
                'decisiontreeclassifier__max_features':['auto', 'sqrt', 'log2'],
                'decisiontreeclassifier__max_depth': [1,2,3,4,5,6,7,8,9,10] }

Trying out different parameters for for GridSearchCV and RandomizedSearchCV for the DecisionTreeClassifier.

In [ ]:
# GridSearchCV
gridsearch = GridSearchCV(pipeline, param_grid)
gridsearch.fit(dicts_Xtrain, Ytrain)
g_acc = accuracy_score(Ytest, gridsearch.predict(dicts_Xtest))
print("Test Accuracy with GridSearch:",g_acc)
print(gridsearch.best_params_)

Test Accuracy with GridSearch: 0.8279589705792028
{'decisiontreeclassifier__criterion': 'gini', 'decisiontreeclassifier__max_depth': 9, 'decisiontreeclassifier__max_features': 'auto', 'decisiontreeclassifier__splitter': 'best'}


In [ ]:
# RandomSearchCV
randomsearch = RandomizedSearchCV(pipeline, param_grid, n_iter=10)
randomsearch.fit(dicts_Xtrain, Ytrain)
r_acc = accuracy_score(Ytest, randomsearch.predict(dicts_Xtest))
print("Test Accuracy with RandomSearch:",r_acc)
print(randomsearch.best_params_)

Test Accuracy with RandomSearch: 0.8409188624777347
{'decisiontreeclassifier__splitter': 'best', 'decisiontreeclassifier__max_features': 'sqrt', 'decisiontreeclassifier__max_depth': 9, 'decisiontreeclassifier__criterion': 'entropy'}


The accuracy from the pipelines used in both grid search and random search differs, but not to a huge extent. Grid search had the highest accuracy of 85% while the random search reached about 84%. This is an improvement from the previous step where the test accuracy reached only 81% where we only used DictVectorizer and not the additional preprocessing steps of StandardScaler and selectKbest.

The GridSearchCV had the higest accuracy with hyperparameters: 

    'splitter': 'best',
    'max_features': 'sqrt',
    'max_depth': 10,
    'criterion': 'gini'}